In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import warnings
import torch
import numpy as np
import nltk
from nltk import sent_tokenize,word_tokenize

warnings.filterwarnings('ignore')

#downloading nltk data
try:
    nltk.download('punkt')
    nltk.download('stopwords')
    nltk.download('punkt_tab')
except:
    pass

#setting device
if torch.cuda.is_available():
    dev='cuda'
else:
    dev='cpu'

device=torch.device(device=dev)
print(f"The code is using device:{device}")

torch.manual_seed(42)
np.random.seed(42)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
The code is using device:cuda


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
# Data loading and pretraining
with open(file=r"/content/drive/MyDrive/wiki.train.tokens", mode="r", encoding='utf-8') as f:
    wiki_text = f.read()
    sentences = sent_tokenize(text=wiki_text)[:40000]
    pre_training_text = [' '.join(sentences[i:i+4]) for i in range(0, len(sentences), 4)]

print(f"Loaded {len(pre_training_text)} pre-training samples")

Loaded 10000 pre-training samples


In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
# tokenizer.bos_token = tokenizer.eos_token
print("Tokenizer initiated")

embedding_model = SentenceTransformer(model_name_or_path='sentence-transformers/all-MiniLM-L6-v2')
print("Embedding model initiated")

Tokenizer initiated
Embedding model initiated


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Config

GPT2Config()

GPT2Config {
  "activation_function": "gelu_new",
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.57.0",
  "use_cache": true,
  "vocab_size": 50257
}

In [ ]:
# GPT-2 pretrained embedding source
gpt2 = GPT2LMHeadModel.from_pretrained("gpt2")

In [ ]:
!pip install pyPDF2

In [ ]:
from PyPDF2 import PdfReader

pdf_path = r"/content/Fed_Meta_Allign.pdf"
reader = PdfReader(pdf_path)

file_text = ""
for page in reader.pages:
    file_text += page.extract_text() + "\n"


In [ ]:
# file_text

In [ ]:
# Process file document for fine-tuning
file_sentences = sent_tokenize(file_text)
finetune_data = [sent.strip() for sent in file_sentences if len(sent) > 10]
print(f"Processed {len(finetune_data)} fine-tuning samples from file document")

# Create document chunks for RAG retrieval
document_chunks = []
paragraphs = file_text.split('\n\n')
for para in paragraphs:
    para = para.strip()
    if len(para) > 40:
        if len(para) > 200:
            sentences = sent_tokenize(para)
            for i in range(0, len(sentences), 2):
                chunk = ' '.join(sentences[i:i+2])
                if len(chunk) > 40:
                    document_chunks.append(chunk)
        else:
            document_chunks.append(para)

print(f"Created {len(document_chunks)} retrieval chunks")
print("Data loading complete!")

Processed 276 fine-tuning samples from file document
Created 142 retrieval chunks
Data loading complete!


In [ ]:
import re

def load_qa_pairs_optimized(file_path):
    """Optimized Q&A loader for your specific format"""
    qa_pairs = []

    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    # Robust regex pattern that handles:
    # - Q1/Q2/Q3 repeating patterns
    # - Multi-line answers
    # - Various spacing formats
    pattern = re.compile(r'Q\d+:\s*(.*?)\s*A\d+:\s*(.*?)(?=\s*Q\d+:|$)', re.DOTALL)

    matches = pattern.findall(content)

    for question, answer in matches:
        # Clean and normalize
        question = ' '.join(question.strip().split())
        answer = ' '.join(answer.strip().split())

        # Quality checks
        if (question and answer and
            len(question) > 10 and  # Reasonable question length
            len(answer) > 20 and    # Reasonable answer length
            not question.lower().startswith('based on') and  # Filter meta-questions
            not answer.lower().startswith('error')):         # Filter errors

            formatted_qa = f"Question: {question}\nAnswer: {answer}"
            qa_pairs.append(formatted_qa)

    # Remove duplicates while preserving order
    seen = set()
    unique_qa_pairs = []
    for qa in qa_pairs:
        if qa not in seen:
            seen.add(qa)
            unique_qa_pairs.append(qa)

    return unique_qa_pairs

# Load and verify Q&A data
print("Loading Q&A pairs...")
qa_data = load_qa_pairs_optimized("filtered_qna.txt")
print(f"Successfully loaded {len(qa_data)} high-quality Q&A pairs")

Loading Q&A pairs...
Successfully loaded 272 high-quality Q&A pairs


In [ ]:
# Show samples
print("\nSample Q&A pairs:")
for i, pair in enumerate(qa_data[:3]):
    print(f"\n--- Sample {i+1} ---")
    print(pair)


Sample Q&A pairs:

--- Sample 1 ---
Question: What is the primary application area highlighted for Fed-Meta-Align?
Answer: The primary application area is real-time fault classification in resource-constrained IoT devices, specifically focusing on industrial safety. This suggests a need for robust models deployed on edge devices.

--- Sample 2 ---
Question: What is a key limitation of standard Federated Learning (FL) that Fed-Meta-Align aims to address?
Answer: Standard FL often struggles with non-IID (non-independent and identically distributed) data. This means data distributions vary significantly across different IoT devices, hindering model performance.

--- Sample 3 ---
Question: What is the primary goal of the Fed-Meta-Align framework as described in the text?
Answer: Fed-Meta-Align aims to address model divergence issues encountered in federated learning scenarios. It does this by establishing a robust initialization and training pipeline for models deployed on IoT devices.


In [ ]:
# Integrate with your SLM training
print(f"\nIntegrating with SLM training...")
print(f"Original fine-tuning samples: {len(finetune_data)}")
enhanced_finetune_data = finetune_data + qa_data
print(f"Enhanced fine-tuning samples: {len(enhanced_finetune_data)}")
print(f"Q&A enrichment: +{len(qa_data)} samples ({len(qa_data)/len(enhanced_finetune_data)*100:.1f}% of total)")


Integrating with SLM training...
Original fine-tuning samples: 276
Enhanced fine-tuning samples: 548
Q&A enrichment: +272 samples (49.6% of total)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2LMHeadModel, GPT2Config
from sklearn.metrics.pairwise import cosine_similarity

!pip install torch_geometric

import subprocess
import sys
from torch_geometric.nn import GCNConv, GATv2Conv
from torch_geometric.data import Data, Batch
print("PyTorch Geometric installed")

PyTorch Geometric installed


In [ ]:
class CGTConfig:
    """Configuration for GNN + Encoder + Decoder"""

    def __init__(self):
        self.vocab_size = 50257
        self.hidden_dim = 768
        self.gnn_layers = 2
        self.transformer_encoder_layers = 4
        self.transformer_decoder_layers = 2
        self.num_heads = 8
        self.gnn_type = 'gat'
        self.dropout = 0.1
        self.max_seq_len = 512

config = CGTConfig()

class GNNEncoderDecoder(nn.Module):
    """ GNN + Encoder + Decoder without fusion"""

    def __init__(self, config):
        super().__init__()
        self.config = config

        # Input embeddings
        self.embedding = nn.Embedding(config.vocab_size, config.hidden_dim)
        self.pos_embedding = nn.Embedding(config.max_seq_len, config.hidden_dim)

        # GNN Layers for local context
        self.gnn_layers = nn.ModuleList()
        for i in range(config.gnn_layers):
            if config.gnn_type == 'gat':
                self.gnn_layers.append(GATv2Conv(config.hidden_dim, config.hidden_dim))
            else:
                self.gnn_layers.append(GCNConv(config.hidden_dim, config.hidden_dim))

        self.gnn_ln = nn.LayerNorm(config.hidden_dim)

        # Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=config.hidden_dim,
            nhead=config.num_heads,
            dim_feedforward=4 * config.hidden_dim,
            dropout=config.dropout,
            batch_first=True
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=config.transformer_encoder_layers)

        # Transformer Decoder
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=config.hidden_dim,
            nhead=config.num_heads,
            dim_feedforward=4 * config.hidden_dim,
            dropout=config.dropout,
            batch_first=True
        )
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers=config.transformer_decoder_layers)

        # Output layer
        self.output_layer = nn.Linear(config.hidden_dim, config.vocab_size)

        # Copy GPT-2 embeddings
        self.embedding.weight.data = gpt2.transformer.wte.weight.data.clone()

        # Tie output projection weights to embeddings
        self.output_layer.weight = self.embedding.weight

        print("✅ GPT-2 embeddings loaded into Corrected CGT model!")


        total_params = sum(p.numel() for p in self.parameters()) / 1e6
        print(f"Corrected Model: {config.gnn_layers} GNN + {config.transformer_encoder_layers} Encoder + {config.transformer_decoder_layers} Decoder")
        print(f"Total parameters: {total_params:.1f}M")

    def create_graph(self, input_ids):
        """Create graph structure from input tokens"""
        batch_size, seq_len = input_ids.shape
        graphs = []

        for b in range(batch_size):
            # Create node features with embeddings + positional encoding
            token_embeds = self.embedding(input_ids[b])
            pos_embeds = self.pos_embedding(torch.arange(seq_len, device=input_ids.device))
            x = token_embeds + pos_embeds

            # Create edges (linear chain with skip connections)
            edge_index = []
            for i in range(seq_len - 1):
                edge_index.append([i, i + 1])  # Forward
                edge_index.append([i + 1, i])  # Backward

            # Skip connections for better information flow
            for i in range(seq_len - 2):
                edge_index.append([i, i + 2])
                edge_index.append([i + 2, i])

            if edge_index:
                edge_index = torch.tensor(edge_index, dtype=torch.long, device=input_ids.device).t().contiguous()
            else:
                edge_index = torch.empty((2, 0), device=input_ids.device, dtype=torch.long)

            graphs.append(Data(x, edge_index))

        return Batch.from_data_list(graphs)

    def encode(self, input_ids, attention_mask=None):
        """Encode input through GNN + Encoder"""
        batch_size, seq_len = input_ids.shape

        # Process through GNN
        graph_batch = self.create_graph(input_ids)
        x_gnn = graph_batch.x

        for gnn_layer in self.gnn_layers:
            x_gnn = F.relu(gnn_layer(x_gnn, graph_batch.edge_index))
            x_gnn = F.dropout(x_gnn, p=self.config.dropout, training=self.training)

        x_gnn = self.gnn_ln(x_gnn)

        # Reshape for transformer
        x_reshaped = x_gnn.view(batch_size, seq_len, -1)

        # Process through encoder
        if attention_mask is not None:
            src_key_padding_mask = ~attention_mask.bool()
        else:
            src_key_padding_mask = None

        memory = self.encoder(x_reshaped, src_key_padding_mask=src_key_padding_mask)
        return memory

    def decode(self, tgt_ids, memory, memory_mask=None, tgt_mask=None):
        """Decode using transformer decoder"""
        batch_size, tgt_len = tgt_ids.shape

        # Prepare target embeddings
        tgt_embeddings = self.embedding(tgt_ids)
        tgt_positions = torch.arange(tgt_len, device=tgt_ids.device).unsqueeze(0).expand(batch_size, -1)
        tgt_embeddings = tgt_embeddings + self.pos_embedding(tgt_positions)

        # Decode
        decoder_output = self.decoder(
            tgt=tgt_embeddings,
            memory=memory,
            tgt_mask=tgt_mask,
            memory_key_padding_mask=memory_mask
        )

        # Output projection
        logits = self.output_layer(decoder_output)
        return logits

    def generate_square_subsequent_mask(self, sz):
        """Generate causal mask for decoder"""
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def forward(self, input_ids, target_ids=None, attention_mask=None):
        """
        Forward pass for training - encoder-decoder with teacher forcing

        """
        # Encode source
        memory = self.encode(input_ids, attention_mask)

        # Use provided target_ids or default to input_ids
        if target_ids is None:
            target_ids = input_ids

        tgt_mask = self.generate_square_subsequent_mask(target_ids.size(1)).to(input_ids.device)

        if attention_mask is not None:
            memory_mask = ~attention_mask.bool()
        else:
            memory_mask = None

        # Decode
        logits = self.decode(target_ids, memory, memory_mask, tgt_mask)
        return logits

# Initialize the model
model = GNNEncoderDecoder(config).to(device)
print(f"Corrected Model initialized with {sum(p.numel() for p in model.parameters())/1e6:.1f}M parameters")

✅ GPT-2 embeddings loaded into Corrected CGT model!
Corrected Model: 2 GNN + 4 Encoder + 2 Decoder
Total parameters: 88.7M
Corrected Model initialized with 88.7M parameters


In [ ]:
def pretrain_language_model(model, data, tokenizer, epochs=None, lr=None, stage_name="Pretraining",batch_size=None):
    """
    General language model pretraining function for learning English patterns

    """
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)
    loss_function = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

    print(f"🚀 {stage_name} for {epochs} epochs with lr={lr}...")
    model.train()

    for epoch in range(epochs):
        total_loss = 0
        batch_count = 0

        batch_size = batch_size

        for i in range(0, len(data), batch_size):
            batch_texts = data[i:i+batch_size]

            # Tokenize with proper special tokens for language modeling
            inputs = tokenizer(
                batch_texts,
                padding=True,
                truncation=True,
                max_length=256,
                return_tensors='pt',
                add_special_tokens=True
            )

            input_ids = inputs['input_ids'].to(device)
            attention_mask = inputs['attention_mask'].to(device)

            # Create proper decoder input (shifted right)
            # For encoder-decoder: decoder sees [tok0, tok1, ..., tokN-1]
            # and predicts [tok1, tok2, ..., tokN]
            decoder_input_ids = input_ids[:, :-1]  # Remove last token for decoder input
            labels = input_ids[:, 1:]  # Remove first token for labels (shifted left)

            # Forward pass with separate source and target
            outputs = model(input_ids, target_ids=decoder_input_ids, attention_mask=attention_mask)

            # Calculate loss - outputs already aligned with labels
            loss = loss_function(
                outputs.contiguous().view(-1, outputs.size(-1)),
                labels.contiguous().view(-1)
            )

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            total_loss += loss.item()
            batch_count += 1

            if batch_count % 10 == 0:
                print(f"{stage_name} Epoch {epoch+1}/{epochs}, Batch {batch_count}, Loss: {loss.item():.4f}")

        avg_loss = total_loss / batch_count if batch_count > 0 else 0
        print(f"✅ {stage_name} Epoch {epoch+1} completed, Average Loss: {avg_loss:.4f}")

    print(f"🎉 {stage_name} completed successfully!")

In [ ]:
print("🚀 Starting Pretraining on English...")
pretrain_language_model(model, pre_training_text, tokenizer, epochs=5, lr=5e-5, stage_name="English Pretraining",batch_size=16)

🚀 Starting Pretraining on English...
🚀 English Pretraining for 5 epochs with lr=5e-05...
English Pretraining Epoch 1/5, Batch 10, Loss: 8.9555
English Pretraining Epoch 1/5, Batch 20, Loss: 8.5515
English Pretraining Epoch 1/5, Batch 30, Loss: 8.9300
English Pretraining Epoch 1/5, Batch 40, Loss: 8.4266
English Pretraining Epoch 1/5, Batch 50, Loss: 8.2238
English Pretraining Epoch 1/5, Batch 60, Loss: 7.7157
English Pretraining Epoch 1/5, Batch 70, Loss: 8.2317
English Pretraining Epoch 1/5, Batch 80, Loss: 8.0523
English Pretraining Epoch 1/5, Batch 90, Loss: 8.1297
English Pretraining Epoch 1/5, Batch 100, Loss: 8.2169
English Pretraining Epoch 1/5, Batch 110, Loss: 8.3675
English Pretraining Epoch 1/5, Batch 120, Loss: 7.3589
English Pretraining Epoch 1/5, Batch 130, Loss: 7.8536
English Pretraining Epoch 1/5, Batch 140, Loss: 7.7360
English Pretraining Epoch 1/5, Batch 150, Loss: 7.7464
English Pretraining Epoch 1/5, Batch 160, Loss: 7.6600
English Pretraining Epoch 1/5, Batch 170

In [ ]:
print("\n📘 Fine-tuning on Domain Paragraphs...")
pretrain_language_model(model, finetune_data, tokenizer, epochs=60, lr=3e-5, stage_name="Domain Fine-tuning",batch_size=8)


📘 Fine-tuning on Domain Paragraphs...
🚀 Domain Fine-tuning for 60 epochs with lr=3e-05...
Domain Fine-tuning Epoch 1/60, Batch 10, Loss: 9.0222
Domain Fine-tuning Epoch 1/60, Batch 20, Loss: 8.7238
Domain Fine-tuning Epoch 1/60, Batch 30, Loss: 7.8060
✅ Domain Fine-tuning Epoch 1 completed, Average Loss: 8.6102
Domain Fine-tuning Epoch 2/60, Batch 10, Loss: 7.3342
Domain Fine-tuning Epoch 2/60, Batch 20, Loss: 7.2530
Domain Fine-tuning Epoch 2/60, Batch 30, Loss: 6.8837
✅ Domain Fine-tuning Epoch 2 completed, Average Loss: 7.1024
Domain Fine-tuning Epoch 3/60, Batch 10, Loss: 6.7322
Domain Fine-tuning Epoch 3/60, Batch 20, Loss: 6.5442
Domain Fine-tuning Epoch 3/60, Batch 30, Loss: 6.4743
✅ Domain Fine-tuning Epoch 3 completed, Average Loss: 6.5581
Domain Fine-tuning Epoch 4/60, Batch 10, Loss: 6.3100
Domain Fine-tuning Epoch 4/60, Batch 20, Loss: 6.2545
Domain Fine-tuning Epoch 4/60, Batch 30, Loss: 6.1312
✅ Domain Fine-tuning Epoch 4 completed, Average Loss: 6.2328
Domain Fine-tunin

In [ ]:
def fine_tune_qa(model, data, tokenizer, epochs=None, lr=None):
    """Fine-tuning function specifically for Q&A task (sequence-to-sequence)"""
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    loss_function = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

    print(f"💬 Instruction Fine-tuning for {epochs} epochs with lr={lr}...")
    model.train()

    for epoch in range(epochs):
        total_loss = 0
        batch_count = 0

        batch_size = 4  # Smaller batch size for Q&A

        for i in range(0, len(data), batch_size):
            batch_qa = data[i:i+batch_size]

            questions = []
            answers = []

            # Parse Q&A pairs
            for qa_pair in batch_qa:
                if "Question:" in qa_pair and "Answer:" in qa_pair:
                    parts = qa_pair.split("Answer:")
                    if len(parts) == 2:
                        q = parts[0].replace("Question:", "").strip()
                        a = parts[1].strip()
                        questions.append(q)
                        answers.append(a)

            if not questions:
                continue

            # Tokenize questions (source) with proper special tokens
            src_inputs = tokenizer(
                questions,
                padding=True,
                truncation=True,
                max_length=256,
                return_tensors='pt',
                add_special_tokens=True
            )

            # Tokenize answers (target) with proper special tokens
            tgt_inputs = tokenizer(
                answers,
                padding=True,
                truncation=True,
                max_length=128,
                return_tensors='pt',
                add_special_tokens=True
            )

            src_ids = src_inputs['input_ids'].to(device)
            src_mask = src_inputs['attention_mask'].to(device)
            tgt_ids = tgt_inputs['input_ids'].to(device)

            # Encode the source (questions)
            memory = model.encode(src_ids, src_mask)

            # Prepare target for decoder (teacher forcing)
            decoder_input_ids = tgt_ids[:, :-1]  # Remove last token for input
            labels = tgt_ids[:, 1:]  # Remove first token for labels

            # Create causal mask for decoder
            tgt_mask = model.generate_square_subsequent_mask(decoder_input_ids.size(1)).to(device)

            # Decode to generate answers
            logits = model.decode(
                decoder_input_ids,
                memory,
                memory_mask=~src_mask.bool(),
                tgt_mask=tgt_mask
            )

            # Use contiguous().view() to avoid stride issues
            logits_flat = logits.contiguous().view(-1, logits.size(-1))
            labels_flat = labels.contiguous().view(-1)

            # Calculate loss
            loss = loss_function(logits_flat, labels_flat)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            total_loss += loss.item()
            batch_count += 1

            if batch_count % 5 == 0:
                print(f" Q&A Epoch {epoch+1}/{epochs}, Batch {batch_count}, Loss: {loss.item():.4f}")

        avg_loss = total_loss / batch_count if batch_count > 0 else 0
        print(f"✅ Q&A Epoch {epoch+1} completed, Average Loss: {avg_loss:.4f}")

    print("🎉 Instruction Fine-tuning completed successfully!")

In [ ]:
print("\n💬 Instruction Fine-tuning (Q&A)...")
fine_tune_qa(model, qa_data, tokenizer, epochs=50, lr=3e-5)

torch.save(model.state_dict(), "/content/corrected_model.pt")
print("💾 Model saved successfully!")



💬 Instruction Fine-tuning (Q&A)...
💬 Instruction Fine-tuning for 50 epochs with lr=3e-05...
 Q&A Epoch 1/50, Batch 5, Loss: 7.5284
 Q&A Epoch 1/50, Batch 10, Loss: 6.2953
 Q&A Epoch 1/50, Batch 15, Loss: 6.1124
 Q&A Epoch 1/50, Batch 20, Loss: 5.7158
 Q&A Epoch 1/50, Batch 25, Loss: 5.6258
 Q&A Epoch 1/50, Batch 30, Loss: 5.4454
 Q&A Epoch 1/50, Batch 35, Loss: 6.1236
 Q&A Epoch 1/50, Batch 40, Loss: 5.7050
 Q&A Epoch 1/50, Batch 45, Loss: 6.0614
 Q&A Epoch 1/50, Batch 50, Loss: 5.7694
 Q&A Epoch 1/50, Batch 55, Loss: 5.6629
 Q&A Epoch 1/50, Batch 60, Loss: 4.7027
 Q&A Epoch 1/50, Batch 65, Loss: 5.2369
✅ Q&A Epoch 1 completed, Average Loss: 5.8832
 Q&A Epoch 2/50, Batch 5, Loss: 5.4667
 Q&A Epoch 2/50, Batch 10, Loss: 4.9427
 Q&A Epoch 2/50, Batch 15, Loss: 5.0980
 Q&A Epoch 2/50, Batch 20, Loss: 5.1653
 Q&A Epoch 2/50, Batch 25, Loss: 5.1196
 Q&A Epoch 2/50, Batch 30, Loss: 4.9039
 Q&A Epoch 2/50, Batch 35, Loss: 5.6513
 Q&A Epoch 2/50, Batch 40, Loss: 5.2211
 Q&A Epoch 2/50, Batch 

In [3]:
print("\nIMPORTANT about our training loss!")
print("   - Pretraining loss should drop below 2.5")
print("   - Domain fine-tuning loss should drop below 2.0")
print("   - Q&A fine-tuning loss should drop below 1.5")
print("   If losses don't drop, we need MORE training data!")


IMPORTANT about our training loss!
   - Pretraining loss should drop below 2.5
   - Domain fine-tuning loss should drop below 2.0
   - Q&A fine-tuning loss should drop below 1.5
   If losses don't drop, we need MORE training data!


In [ ]:
class RAGSystem:
    """RAG system - Each query is independent, with intelligent output extraction"""

    def __init__(self, model, tokenizer, embedding_model, document_chunks):
        self.model = model
        self.tokenizer = tokenizer
        self.embedding_model = embedding_model
        self.document_chunks = document_chunks

        print("Computing embeddings for chunks...")
        self.chunk_embeddings = self.embedding_model.encode(document_chunks)
        print(f"Embeddings computed for {len(document_chunks)} chunks")

    def retrieve_relevant_chunks(self, query, top_k=3):
        """Retrieve relevant document chunks"""
        query_embedding = self.embedding_model.encode([query])
        similarities = cosine_similarity(query_embedding, self.chunk_embeddings)
        top_indices = np.argsort(similarities[0])[-top_k:][::-1]

        relevant_chunks = [self.document_chunks[i] for i in top_indices]
        return relevant_chunks

    def build_prompt(self, query, context_chunks):
        """
        Build prompt WITHOUT conversation history (independent queries)
        """
        context = "\n".join([f"- {chunk[:300]}" for chunk in context_chunks[:2]])

        #  Simple, direct prompt without history
        prompt = f"""Based on the following context, provide a clear and accurate answer to the question.

Context:
{context}

Question: {query}
Answer:"""

        return prompt

    def extract_answer_intelligently(self, raw_output, query):
        """
        Intelligent answer extraction from model output
        """
        if not raw_output or len(raw_output.strip()) == 0:
            return None

        # Remove common prefixes that models sometimes generate
        prefixes_to_remove = [
            "Answer:",
            "answer:",
            "A:",
            "Based on the context,",
            "According to the context,",
            "The answer is",
            "the answer is"
        ]

        cleaned = raw_output.strip()
        for prefix in prefixes_to_remove:
            if cleaned.startswith(prefix):
                cleaned = cleaned[len(prefix):].strip()

        # If output is just repeated tokens or gibberish, return None
        if self.is_gibberish(cleaned):
            return None

        # Extract first complete sentence or paragraph
        sentences = sent_tokenize(cleaned)

        if not sentences:
            return None

        # Find first meaningful sentence (not too short, not gibberish)
        meaningful_sentences = []
        for sent in sentences:
            if len(sent.split()) >= 5 and not self.is_gibberish(sent):
                meaningful_sentences.append(sent)

            # Stop after 3-4 good sentences for concise answers
            if len(meaningful_sentences) >= 3:
                break

        if not meaningful_sentences:
            return None

        answer = ' '.join(meaningful_sentences)

        # Final cleanup
        answer = self.final_cleanup(answer)

        return answer if len(answer.split()) >= 5 else None

    def is_gibberish(self, text):
        """
        Detect if text is gibberish
        """
        if not text or len(text.strip()) < 5:
            return True

        # Check for excessive special characters
        special_char_ratio = sum(1 for c in text if not c.isalnum() and not c.isspace()) / len(text)
        if special_char_ratio > 0.3:
            return True

        # Check for repeated short patterns (like "Al Al Al")
        words = text.split()
        if len(words) > 3:
            # Check if too many repeated words
            unique_words = set(words)
            if len(unique_words) / len(words) < 0.3:
                return True

        # Check for excessive single-letter words
        single_letters = sum(1 for w in words if len(w) == 1)
        if len(words) > 0 and single_letters / len(words) > 0.3:
            return True

        # Check for unknown characters (�)
        if '�' in text or '\ufffd' in text:
            return True

        return False

    def final_cleanup(self, text):
        """
        Final text cleanup
        """
        # Remove extra whitespace
        text = ' '.join(text.split())

        # Ensure proper capitalization
        if text and text[0].islower():
            text = text[0].upper() + text[1:]

        # Ensure proper ending punctuation
        if text and not text.endswith(('.', '!', '?')):
            text += '.'

        return text

    def generate_response(self, query, max_length=150):
        """
        Generation with intelligent extraction
        """
        # Retrieve context
        relevant_chunks = self.retrieve_relevant_chunks(query, top_k=2)

        # Build prompt (without history)
        prompt = self.build_prompt(query, relevant_chunks)

        # Tokenize prompt
        inputs = self.tokenizer(
            prompt,
            return_tensors='pt',
            max_length=384,
            truncation=True,
            padding=True,
            add_special_tokens=True
        )

        source_ids = inputs['input_ids'].to(device)
        source_mask = inputs['attention_mask'].to(device)

        # Generate with model
        self.model.eval()
        with torch.no_grad():
            memory = self.model.encode(source_ids, source_mask)

            # Start generation
            generated_ids = torch.tensor([[self.tokenizer.pad_token_id]], device=device)

            for step in range(max_length):
                tgt_mask = self.model.generate_square_subsequent_mask(generated_ids.size(1)).to(device)

                logits = self.model.decode(
                    generated_ids,
                    memory,
                    memory_mask=~source_mask.bool(),
                    tgt_mask=tgt_mask
                )

                next_token_logits = logits[:, -1, :]

                # Sampling parameters
                temperature = 0.7
                top_k = 50

                next_token_logits = next_token_logits / temperature

                # Top-k filtering
                indices_to_remove = next_token_logits < torch.topk(next_token_logits, top_k)[0][..., -1, None]
                next_token_logits[indices_to_remove] = -float('Inf')

                # Repetition penalty
                if generated_ids.size(1) > 10:
                    repetition_penalty = 1.3
                    recent_tokens = set(generated_ids[0, -15:].tolist())
                    for token in recent_tokens:
                        if next_token_logits[0, token] > 0:
                            next_token_logits[0, token] /= repetition_penalty
                        else:
                            next_token_logits[0, token] *= repetition_penalty

                # Sample next token
                probs = F.softmax(next_token_logits, dim=-1)
                next_token_id = torch.multinomial(probs, num_samples=1)

                # Stop conditions
                if next_token_id.item() in [self.tokenizer.eos_token_id, self.tokenizer.pad_token_id]:
                    break

                generated_ids = torch.cat([generated_ids, next_token_id], dim=1)

                # Early stopping for complete answers
                if step > 30:
                    current_text = self.tokenizer.decode(generated_ids[0], skip_special_tokens=True)
                    if len(current_text.split()) > 20 and any(p in current_text[-15:] for p in ['.', '!', '?']):
                        break

            # Decode response
            raw_response = self.tokenizer.decode(generated_ids[0], skip_special_tokens=True)

            # Intelligent extraction
            extracted_answer = self.extract_answer_intelligently(raw_response, query)

            # Fallback to context-based answer if model output is poor
            if extracted_answer is None or len(extracted_answer.split()) < 5:
                extracted_answer = self.fallback_to_context(query, relevant_chunks)

            return extracted_answer, relevant_chunks

    def fallback_to_context(self, query, context_chunks):
        """
        Fallback strategy - extract relevant sentences from context
        """
        if not context_chunks:
            return "I don't have enough information to answer this question based on the provided context."

        # Simple extractive approach: find most relevant sentences
        query_words = set(query.lower().split())
        best_sentences = []

        for chunk in context_chunks[:2]:
            sentences = sent_tokenize(chunk)
            for sent in sentences:
                sent_words = set(sent.lower().split())
                overlap = len(query_words & sent_words)
                if overlap >= 2 and len(sent.split()) >= 5:
                    best_sentences.append((overlap, sent))

        if best_sentences:
            # Sort by overlap and take top 2-3 sentences
            best_sentences.sort(reverse=True, key=lambda x: x[0])
            answer = ' '.join([sent for _, sent in best_sentences[:2]])
            return self.final_cleanup(answer)

        return "I don't have enough information to answer this question based on the provided context."

    def chat(self, user_input):
        """
        Independent query processing (no history)
        """
        response, sources = self.generate_response(user_input)

        return {
            "response": response,
            "sources": sources[:2]
        }

# Initialize the improved RAG system
print("\n" + "="*60)
print("Initializing Improved RAG System (Independent Queries)...")
print("="*60)
rag_system = RAGSystem(model, tokenizer, embedding_model, document_chunks)
print("✅ Improved RAG System ready!")


Initializing Improved RAG System (Independent Queries)...
Computing embeddings for chunks...
Embeddings computed for 142 chunks
✅ Improved RAG System ready!


In [1]:
# Test the system
print("=== RAG SYSTEM TESTING ===")

test_questions = [
    "What is Federated Learning?",
    "what is Fed_Meta Allign?"
]

for i, question in enumerate(test_questions, 1):
    print(f"\nQ{i}: {question}")
    result = rag_system.chat(question)
    print(f"Answer: {result['response']}")

    # if result['sources']:
    #     print("Sources used:")
    #     for j, source in enumerate(result['sources'], 1):
    #         print(f"  {j}. {source[:100]}...")

 
=== CORRECTED RAG SYSTEM TEST ===

Q1: What is Federated Learning?
Answer: A distributed machine learning approach where multiple devices collaboratively train a global model while keeping their local data private.

Q2: What is Fed-Meta-Align?
Answer: A federated TinyML framework that aligns model updates across heterogeneous devices using similarity-aware aggregation and personalization.


